In [1]:
%load_ext autoreload
%autoreload 2 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from src.data_processing.pipelines.ClassifierPipe import ClassifierPipe

import tensorflow as tf

2023-08-31 14:37:50.018183: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 14:37:51.116319: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 14:38:29.090406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
# print gpu available to tensorflow
print(f"Number of GPUs available: {len(tf.config.list_physical_devices('GPU'))}")

Number of GPUs available: 2


In [ ]:
DATA_PATH = '/projects/p31961/gaby_data/aggregated_data/raw_data/datasets/raw_data_raw_data.parquet.gzip'

In [4]:
processor_pipe = (ClassifierPipe(DATA_PATH)
             .read_raw_data()
             .calculate_max_min_signal()
             .calculate_percent_avoid()
             .drop_columns(["event", "action", "trial", "trial_count", "num_avoids", "max_trial"])
             .split_data(test_size=0.3,
                test_dev_size=0.5, 
                split_group = "mouse_id", 
                stratify_group = "sex", 
                target='ratio_avoid',
                save_subject_ids=False)
                # path_to_save =os.path.dirname(raw_path)
            .transorm_data()
)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
model.fit(processor_pipe.X_train, processor_pipe.y_train, validation_data=(processor_pipe.X_dev, processor_pipe.y_dev),epochs=100)


In [ ]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])

In [9]:
num_cols = processor_pipe.processor.named_transformers_["num"].get_feature_names_out().tolist()
cat_cols = processor_pipe.processor.named_transformers_["cat"].get_feature_names_out().tolist()
cols = num_cols + cat_cols

testing_df = (pd.DataFrame(processor_pipe.X_test, columns=cols)
              .assign(predicted_avoid_ratio = model.predict(processor_pipe.X_test), 
                #       true_avoid_ratio = processor_pipe.y_test.values.reshape(-1,1)
                      )
)
dopamine = testing_df.query("sensor_DA ==1").drop(columns = ["sex_M"]).drop_duplicates(subset = ["mouse_id", "day"])
sns.scatterplot(data=dopamine, x="day", y="true_avoid_ratio")
sns.scatterplot(data=dopamine, x="day", y="predicted_avoid_ratio", alpha=0.3)

NameError: name 'model' is not defined

In [25]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', 
                             n_estimators=60, max_depth = 6,
                             learning_rate = 0.1,
                             max_leaf_nodes = 3, 
                             eval_metric=['rmse', 'mae'])
xgb_model.fit(processor_pipe.X_train, processor_pipe.y_train, eval_set = [(processor_pipe.X_dev, processor_pipe.y_dev)])
xg_pred = xgb_model.predict(processor_pipe.X_test)



[16:41:51] WARNING: ../src/learner.cc:767: 
Parameters: { "max_leaf_nodes" } are not used.

[0]	validation_0-rmse:0.19440	validation_0-mae:0.14454
[1]	validation_0-rmse:0.18833	validation_0-mae:0.14406
[2]	validation_0-rmse:0.18636	validation_0-mae:0.14615
[3]	validation_0-rmse:0.18900	validation_0-mae:0.15413
[4]	validation_0-rmse:0.19213	validation_0-mae:0.16361
[5]	validation_0-rmse:0.19831	validation_0-mae:0.17453
[6]	validation_0-rmse:0.20462	validation_0-mae:0.18333
[7]	validation_0-rmse:0.21159	validation_0-mae:0.19313
[8]	validation_0-rmse:0.21847	validation_0-mae:0.20171
[9]	validation_0-rmse:0.22564	validation_0-mae:0.21004
[10]	validation_0-rmse:0.23227	validation_0-mae:0.21721
[11]	validation_0-rmse:0.23793	validation_0-mae:0.22294
[12]	validation_0-rmse:0.24341	validation_0-mae:0.22844
[13]	validation_0-rmse:0.24845	validation_0-mae:0.23347
[14]	validation_0-rmse:0.25345	validation_0-mae:0.23817
[15]	validation_0-rmse:0.25826	validation_0-mae:0.24249
[16]	validation_0-rmse

In [22]:
xgb_model.score(processor_pipe.X_test, processor_pipe.y_test)

0.4418642245338793

In [30]:
metrics = xgb_model.evals_result()
metrics.keys()
# plt.plot(rmse)

dict_keys(['validation_0'])

In [ ]:
testing_df = testing_df.assign(xg_pred_avoid_ratio = xg_pred)
dopamine = testing_df.query("sensor_DA ==1").drop(columns = ["sex_M"]).drop_duplicates(subset = ["mouse_id", "day"])
sns.scatterplot(data=testing_df, x="day", y="true_avoid_ratio")
sns.scatterplot(data=testing_df, x="day", y="xg_pred_avoid_ratio", alpha=0.1)

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

In [ ]:
space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),

}
trials = Trials()
def obejective(params):
    max_depth = params['max_depth']
    # n_estimators = params['n_estimators']
    # learning_rate = params['learning_rate']
    # min_child_weight = params['min_child_weight']
    
    xg_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth = max_depth, eval_metric=['rmse', 'mae'])
    model = xg_model.fit(processor_pipe.X_train, processor_pipe.y_train)
    return mean_squared_error(processor_pipe.y_test, model.predict(processor_pipe.X_test))

fmin(obejective, space, algo=tpe.suggest, max_evals=100, trials=trials)

